In [4]:
import pandas as pd
import backtesting as bt
import pandas_ta as ta
import numpy as np
import datetime as dt
from breeze_connect import BreezeConnect



In [36]:
# prompt: create a function which will return two variables start_date and end_date. function will two parameter period = which is the difference between start date and end date. period value can be 1y for years, 1m for months, 1d for days,,1h for hours. second parameter latest_date = which has default value of current date.

def date_range(period, latest_date=dt.date.today()):
  """
  Returns a tuple of start_date and end_date based on the provided period and latest_date.

  Args:
    period: A string representing the period. Valid values are "1y", "1m", "1d", and "1h".
    latest_date: A datetime.date object representing the latest date. Defaults to today's date.

  Returns:
    A tuple of (start_date, end_date).
  """
  num = int(period[:-1])
  period = period[-1]
  #print(num,period)

  if period == "y":
    start_date = latest_date - dt.timedelta(days=num*365)
  elif period == "m":
    start_date = latest_date - dt.timedelta(days=num*30)
  elif period == "d":
    start_date = latest_date - dt.timedelta(days=num)
  elif period == "h":
    start_date = latest_date - dt.timedelta(hours=num)
  else:
    raise ValueError("Invalid period value. Should in format interger followed by [y,m,d,h]")

  end_date = latest_date

  return start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')

def download(symbol,interval,period,exchange_code='NSE',product_type='cash',expiry_date=None,right=None,strike_price =None):

  #values check
  interval_list = ["1second","1minute","5minute","30minute","1day"]
  if interval not in interval_list:
    raise ValueError('Interval should be from "1second","1minute","5minute","30minute","1day"')

  exchange_code_list  = ["NSE", "NFO", "BSE","NDX","MCX"]
  if exchange_code not in exchange_code_list:
    raise ValueError('Exchange code should be from "NSE", "NFO", "BSE","NDX","MCX"')

  product_type_list = 	['cash', 'options',"futures"]
  if product_type not in product_type_list:
    raise ValueError('product_type should be Cash, Options,Futures (Required for NFO, NDX,MCX)' )

  if exchange_code == 'NFO':
    if expiry_date == None:
      raise ValueError('Expiry Date is required if exchange_code=NFO. Required if exchange code is NFO, NDX,MCX')

    if product_type == 'cash':
      raise ValueError('product_type is required if exchange_code = NFO. product_type can not be cash in case of exchange_code = NFO')

    if expiry_date!=None:
    
      signal = False
      try:
        day, month, year = map(int, expiry_date.split('-'))

        # Check for valid day, month, and year ranges
        if 1 <= day <= 31 and 1 <= month <= 12 and year >= 2000:
          signal = True
        
        else:
          raise ValueError(f'Enter a Valid Date in format yyyy-mm-dd. Entered Date =>  Day : {day}, Month : {month}, Year : {year}')
      except ValueError:
        raise ValueError(f"Enter date in Format dd-mm-yyyy. All dates should be in Number seprated by '-'.")
      
      if signal:
        expiry_date =  dt.date(year=year,month=month,day=day)
        expiry_date = expiry_date.strftime('%Y-%m-%d')
        print(expiry_date)

    if right == None:
      raise ValueError('"right" is rquired if exchange_code is NFO,NDX, MCX')
    
    else:
      right_list = ['call','put','others']
      if right not in right_list:
        raise ValueError('"right" should be from [call,put,others]')
    
    if strike_price == None:
      raise ValueError('"strike_price" is rquired if exchange_code is NFO,NDX, MCX. Enter "0" in case of future or where strike is not required in case product_type = NFO')
    
    else:
      try:
        strike = int(strike_price)
      except Exception as e:
        raise ValueError('strike_price should be numeric value in "" for example "5000"')
      













  start_date, end_date = date_range(period)

  if exchange_code=='NSE':

    result = breeze.get_historical_data_v2(interval=interval,
                              from_date= f"{start_date}T07:00:00.000Z",
                              to_date= f"{end_date}T07:00:00.000Z",
                              stock_code= symbol,
                              exchange_code= exchange_code,
                              product_type=product_type,
                              )

  elif exchange_code == 'NFO':

    result = breeze.get_historical_data_v2(interval=interval,
                            from_date= f"{start_date}T07:00:00.000Z",
                            to_date= f"{end_date}T07:00:00.000Z",
                            stock_code=symbol,
                            exchange_code=exchange_code,
                            product_type=product_type,
                            expiry_date=f"{expiry_date}T07:00:00.000Z",
                            right=right,
                            strike_price=strike_price)





  if result['Error'] == None and result['Status']==200:
      data = result['Success']
  else:
      raise ValueError(result['Error'])

  return pd.DataFrame(data)

data = download('ITC','5minute','5d',exchange_code='NFO',expiry_date='25-04-2024',product_type='futures',right='others',strike_price= '0')

print(data)


2024-04-25
      close             datetime exchange_code  expiry_date    high     low  \
0    419.20  2024-04-19 09:15:00           NFO  25-APR-2024  419.80  416.00   
1    421.00  2024-04-19 09:20:00           NFO  25-APR-2024  422.55  419.15   
2    421.80  2024-04-19 09:25:00           NFO  25-APR-2024  422.20  420.55   
3    422.10  2024-04-19 09:30:00           NFO  25-APR-2024  422.35  421.05   
4    422.25  2024-04-19 09:35:00           NFO  25-APR-2024  422.80  422.00   
..      ...                  ...           ...          ...     ...     ...   
220  428.85  2024-04-23 15:05:00           NFO  25-APR-2024  429.30  428.80   
221  428.65  2024-04-23 15:10:00           NFO  25-APR-2024  428.90  428.50   
222  428.55  2024-04-23 15:15:00           NFO  25-APR-2024  428.95  428.40   
223  428.70  2024-04-23 15:20:00           NFO  25-APR-2024  428.70  428.40   
224  428.40  2024-04-23 15:25:00           NFO  25-APR-2024  428.60  428.20   

       open  open_interest product_type 

In [5]:
from breeze_connect import BreezeConnect

key = '73=63fT44781tY4C42L0i3491201v23A'
session_code = '39319109'
secret_key = '11331XF5fm6c5&Y3K2Q99lp28347480+'

# Initialize SDK
breeze = BreezeConnect(api_key=key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=73=63fT44781tY4C42L0i3491201v23A
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.

# Generate Session
breeze.generate_session(api_secret=secret_key,
                        session_token=session_code)

# Generate ISO8601 Date/DateTime String
'''
import datetime
iso_date_string = datetime.datetime.strptime("28/02/2021","%d/%m/%Y").isoformat()[:10] + 'T05:30:00.000Z'
iso_date_time_string = datetime.datetime.strptime("28/02/2021 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + '.000Z'
'''

'\nimport datetime\niso_date_string = datetime.datetime.strptime("28/02/2021","%d/%m/%Y").isoformat()[:10] + \'T05:30:00.000Z\'\niso_date_time_string = datetime.datetime.strptime("28/02/2021 23:59:59","%d/%m/%Y %H:%M:%S").isoformat()[:19] + \'.000Z\'\n'

In [18]:
interval = '1minute'   #options = 1minute    1day
start_date, end_date = date_range('5d')

result = breeze.get_historical_data_v2(interval="5minute",
                            from_date= f"{start_date}T07:00:00.000Z",
                            to_date= f"{end_date}T07:00:00.000Z",
                            stock_code="ICIBAN",
                            exchange_code="NSE",
                            product_type="cash",
                            )

result

{'Error': None,
 'Status': 200,
 'Success': [{'close': 1053.0,
   'datetime': '2024-04-19 09:05:00',
   'exchange_code': 'NSE',
   'high': 1053.0,
   'low': 1053.0,
   'open': 1053.0,
   'stock_code': 'ICIBAN',
   'volume': 223562},
  {'close': 1053.05,
   'datetime': '2024-04-19 09:15:00',
   'exchange_code': 'NSE',
   'high': 1053.9,
   'low': 1048.1,
   'open': 1048.1,
   'stock_code': 'ICIBAN',
   'volume': 637209},
  {'close': 1053.25,
   'datetime': '2024-04-19 09:20:00',
   'exchange_code': 'NSE',
   'high': 1055.8,
   'low': 1051.3,
   'open': 1053.2,
   'stock_code': 'ICIBAN',
   'volume': 272497},
  {'close': 1055.5,
   'datetime': '2024-04-19 09:25:00',
   'exchange_code': 'NSE',
   'high': 1055.5,
   'low': 1052.75,
   'open': 1053.75,
   'stock_code': 'ICIBAN',
   'volume': 115757},
  {'close': 1054.35,
   'datetime': '2024-04-19 09:30:00',
   'exchange_code': 'NSE',
   'high': 1056.95,
   'low': 1053.4,
   'open': 1055.5,
   'stock_code': 'ICIBAN',
   'volume': 173412},
 

In [ ]:
data = ''
if result['Error'] == None and result['Status']==200:
  data = result['Success']
else:
  raise result['Error']

{'Error': None,
 'Status': 200,
 'Success': [{'close': 1075.0,
   'datetime': '2024-04-18 09:07:00',
   'exchange_code': 'NSE',
   'high': 1075.0,
   'low': 1075.0,
   'open': 1075.0,
   'stock_code': 'ICIBAN',
   'volume': 234051},
  {'close': 1076.35,
   'datetime': '2024-04-18 09:15:00',
   'exchange_code': 'NSE',
   'high': 1078.05,
   'low': 1072.8,
   'open': 1078.05,
   'stock_code': 'ICIBAN',
   'volume': 296472},
  {'close': 1075.0,
   'datetime': '2024-04-18 09:16:00',
   'exchange_code': 'NSE',
   'high': 1075.6,
   'low': 1074.05,
   'open': 1075.55,
   'stock_code': 'ICIBAN',
   'volume': 106237},
  {'close': 1074.95,
   'datetime': '2024-04-18 09:17:00',
   'exchange_code': 'NSE',
   'high': 1074.95,
   'low': 1074.3,
   'open': 1074.7,
   'stock_code': 'ICIBAN',
   'volume': 82409},
  {'close': 1074.6,
   'datetime': '2024-04-18 09:18:00',
   'exchange_code': 'NSE',
   'high': 1074.95,
   'low': 1073.65,
   'open': 1074.75,
   'stock_code': 'ICIBAN',
   'volume': 41477},